https://learn.activeloop.ai/courses/take/langchain/multimedia/46317643-langchain-101-from-zero-to-hero

Before executing the following code, make sure to have
your OpenAI key saved in the “OPENAI_API_KEY” environment variable.


In [2]:
!pip install python-dotenv

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
# print(os.getenv("ACTIVELOOP_TOKEN"))
# print(os.getenv("OPENAI_API_KEY"))

True

In [4]:
from langchain.llms import OpenAI

llm = OpenAI(model="text-davinci-003", temperature=0.9)
text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."
print(llm(text))



1. Morning jog: Start off each morning with a 45-minute jog focusing on an endurance pace. Increase the time and distance by 10 minutes each week.

2. Afternoon bike ride: Go on a bike ride for 45 minutes in the afternoon. Focus on distance over speed. Increase the duration of the ride by 10 minutes each week as your endurance improves. 

3. Evening walk: Spend 30 minutes walking in the evening. Focus on walking at a brisk pace. Increase the distance by 10 minutes each week. 

4. Interval training: Use interval training every other day. Run or bike (depending on the weather) for 5 minutes at a moderate pace and 2 minutes at a faster pace. Repeat this cycle for 30 minutes. Increase the intensity and duration as your endurance improves. 

5. Core workouts: Complete core workouts such as planks, mountain climbers and side walks at least 3 times a week. Focus on strengthening your core and increasing your cardiovascular endurance. 

6. Resistance training: Incorporate resistance training

In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# instantiate the LLM and embeddings models
llm = OpenAI(model="text-davinci-003", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# Create Deep Lake dataset
# Use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "mpazaryna"
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Deep Lake Dataset in hub://mpazaryna/langchain_course_from_zero_to_hero already exists, loading from the storage


Creating 2 embeddings in 1 batches of size 2:: 100%|██████████| 1/1 [00:05<00:00,  5.22s/it]

Dataset(path='hub://mpazaryna/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (42, 1536)  float32   None   
    id        text      (42, 1)      str     None   
 metadata     json      (42, 1)      str     None   
   text       text      (42, 1)      str     None   


['77c164b2-a56f-11ee-87b9-000d3a5341f9',
 '77c1662e-a56f-11ee-87b9-000d3a5341f9']

In [6]:
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

In [7]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [8]:
response = agent.run("When was Napoleone born?")
print(response)



> Entering new  chain...
 I need to find out when Napoleone was born.
Action: Retrieval QA System
Action Input: When was Napoleone born?
Observation:  Napoleon Bonaparte was born on 15 August 1769.
Thought: I now know the final answer.
Final Answer: Napoleon Bonaparte was born on 15 August 1769.

> Finished chain.
Napoleon Bonaparte was born on 15 August 1769.


In [9]:
# load the existing Deep Lake dataset and specify the embedding function
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# create new documents
texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Deep Lake Dataset in hub://mpazaryna/langchain_course_from_zero_to_hero already exists, loading from the storage


Creating 2 embeddings in 1 batches of size 2:: 100%|██████████| 1/1 [00:04<00:00,  4.10s/it]

Dataset(path='hub://mpazaryna/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (44, 1536)  float32   None   
    id        text      (44, 1)      str     None   
 metadata     json      (44, 1)      str     None   
   text       text      (44, 1)      str     None   


['87f86e2a-a56f-11ee-87b9-000d3a5341f9',
 '87f86f7e-a56f-11ee-87b9-000d3a5341f9']

In [10]:
# instantiate the wrapper class for GPT3
llm = OpenAI(model="text-davinci-003", temperature=0)

# create a retriever from the db
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm, chain_type="stuff", retriever=db.as_retriever()
)

# instantiate a tool that uses the retriever
tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

# create an agent that uses the tool
agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [11]:
response = agent.run("When was Michael Jordan born?")
print(response)



> Entering new  chain...
 I need to find out when Michael Jordan was born.
Action: Retrieval QA System
Action Input: When was Michael Jordan born?
Observation:  Michael Jordan was born on 17 February 1963.
Thought: I now know the final answer.
Final Answer: Michael Jordan was born on 17 February 1963.

> Finished chain.
Michael Jordan was born on 17 February 1963.
